In [25]:
import numpy as np
from scipy import stats
from dataclasses import dataclass

In [56]:
@dataclass
class Sample:
    array: np.ndarray
    ddof: int = 1
    mean: float = None
    std: float = None

    def __post_init__(self):
        self.mean, self.std = self.get_basic_stats()

    def get_basic_stats(self) -> tuple[float, float]:
        mean = np.mean(self.array)
        std = np.std(self.array, ddof=self.ddof)
        return mean, std

    def __repr__(self):
        return f"mean: {self.mean}, std: {self.std}"

In [57]:
def print_result(x: Sample, formatter: str = ".4f") -> None:
    print("Sample mean: ", format(x.mean, formatter))
    print("Sample std: ", format(x.std, formatter))

In [62]:
def compare_two_samples(s1: Sample, s2: Sample) -> tuple[float, float]:
    """Return t-statistic and p-value of two samples."""
    t_stat, p_value = stats.ttest_ind(s1.array, s2)  # just to check; fake!
    return t_stat, p_value

In [63]:
step_7_from_scratch_fcd = np.array(
    [
        0.2227403282428213,
        0.2264341671785246,
        0.22185662044498144,
        0.22738946004770355,
        0.21581233327245286,
    ]
)
sample = Sample(step_7_from_scratch_fcd)
print_result(sample)

Sample mean:  0.2228
Sample std:  0.0046


In [71]:
s1 = sample.array
s2 = s1 * 1.05
print(compare_two_samples(s1, s2))

(-3.750021931871041, 0.00562405456328571)
